# Machine Unlearning: All Methods Comparison

This notebook runs **all three unlearning methods** on **both architectures** (ResNet-18 and VGG-16-BN) for a comprehensive comparison.

## Methods Implemented:
1. **Random Labeling** - Naive baseline: assigns random labels to forget class samples
2. **Gradient Ascent** - Targeted forgetting: maximizes loss on forget class
3. **SalUn** - Saliency-based: updates only the most important weights for forgetting

## Setup Instructions for Google Colab:
1. Upload `unlearning_utils.py` to the Files panel (left sidebar)
2. Run all cells in order
3. Results will be saved to `notebook_results/` directory

In [ ]:
# Install required packages
!pip install -q timm umap-learn

# Setup path to find unlearning_utils.py (uploaded to /content/)
import sys
if '/content' not in sys.path:
    sys.path.append('/content')

# Verify the file exists
import os
if not os.path.exists('/content/unlearning_utils.py'):
    print("WARNING: Please upload unlearning_utils.py using the Files tab")
else:
    print("Setup complete - unlearning_utils.py found")

In [ ]:
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from typing import Dict

from unlearning_utils import (
    get_resnet18, get_vgg16bn, get_data_loaders, get_umap_subset,
    create_results_json, save_results, SEED
)

# Set random seeds
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# ============================================================================
# Configuration
# ============================================================================

FORGET_CLASS = 0        # Class to unlearn (0-9 for CIFAR-10)
EPOCHS = 5              # Number of unlearning epochs
BATCH_SIZE = 128        # Batch size
LEARNING_RATE = 0.1     # Learning rate
MOMENTUM = 0.9          # SGD momentum
WEIGHT_DECAY = 5e-4     # Weight decay
NUM_CLASSES = 10        # CIFAR-10 classes

# Method-specific parameters
MAX_GRAD_NORM = 100.0      # Gradient clipping for Gradient Ascent
SALIENCY_THRESHOLD = 0.75  # Top 75% weights for SalUn

# Device selection
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")
print(f"Forget class: {FORGET_CLASS}")
print(f"Epochs: {EPOCHS}, Batch size: {BATCH_SIZE}, LR: {LEARNING_RATE}")

## Unlearning Method Implementations

In [ ]:
# ============================================================================
# Method 1: Random Labeling
# ============================================================================

def random_labeling_unlearn(
    model: nn.Module,
    retain_loader: DataLoader,
    forget_loader: DataLoader,
    forget_class: int,
    epochs: int,
    lr: float,
    device: torch.device,
    momentum: float = 0.9,
    weight_decay: float = 5e-4
) -> nn.Module:
    """
    Random Labeling Unlearning Method.
    Combines retain and forget data. For forget class samples,
    assigns random labels from remaining classes.
    """
    combined_dataset = ConcatDataset([retain_loader.dataset, forget_loader.dataset])
    combined_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    remain_classes = [i for i in range(NUM_CLASSES) if i != forget_class]
    
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in combined_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Assign random labels to forget class
            forget_mask = (labels == forget_class)
            if forget_mask.sum() > 0:
                random_labels = torch.tensor([
                    remain_classes[torch.randint(0, len(remain_classes), (1,)).item()]
                    for _ in range(forget_mask.sum())
                ], device=device)
                labels[forget_mask] = random_labels
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"  Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(combined_loader):.4f}")
    
    return model

In [ ]:
# ============================================================================
# Method 2: Gradient Ascent
# ============================================================================

def gradient_ascent_unlearn(
    model: nn.Module,
    forget_loader: DataLoader,
    epochs: int,
    lr: float,
    device: torch.device,
    max_grad_norm: float = 100.0,
    momentum: float = 0.9,
    weight_decay: float = 5e-4
) -> nn.Module:
    """
    Gradient Ascent Unlearning Method.
    Trains ONLY on forget data using NEGATIVE cross-entropy loss.
    """
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in forget_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = -criterion(outputs, labels)  # NEGATIVE loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            running_loss += (-loss.item())
        
        print(f"  Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(forget_loader):.4f}")
    
    return model

In [ ]:
# ============================================================================
# Method 3: SalUn (Saliency-based Unlearning)
# ============================================================================

def compute_gradient_saliency(
    model: nn.Module,
    forget_loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
    threshold: float = 0.75,
    max_batches: int = 5
) -> Dict[str, torch.Tensor]:
    """Compute gradient-based weight saliency mask."""
    print("  Computing gradient-based weight saliency...")
    
    gradient_dict = {}
    for name, param in model.named_parameters():
        if param.requires_grad:
            gradient_dict[name] = torch.zeros_like(param)
    
    model.eval()
    batch_count = 0
    
    for inputs, labels in forget_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        outputs = model(inputs)
        loss = -criterion(outputs, labels)
        loss.backward()
        
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                gradient_dict[name] += param.grad.abs()
        
        batch_count += 1
        if batch_count >= max_batches:
            break
    
    for name in gradient_dict:
        gradient_dict[name] /= batch_count
    
    all_grads = torch.cat([gradient_dict[name].flatten() for name in gradient_dict])
    k = int(threshold * len(all_grads))
    threshold_value = torch.topk(all_grads, k)[0][-1] if k > 0 else float('inf')
    
    mask = {name: (gradient_dict[name] >= threshold_value).float().to(device) for name in gradient_dict}
    
    total_params = sum(m.numel() for m in mask.values())
    selected_params = sum(m.sum().item() for m in mask.values())
    print(f"  Saliency mask: {int(selected_params):,}/{total_params:,} params ({selected_params/total_params*100:.1f}%)")
    
    return mask


def salun_unlearn(
    model: nn.Module,
    retain_loader: DataLoader,
    forget_loader: DataLoader,
    forget_class: int,
    epochs: int,
    lr: float,
    device: torch.device,
    saliency_threshold: float = 0.75,
    grad_clip: float = 100.0,
    momentum: float = 0.9,
    weight_decay: float = 5e-4
) -> nn.Module:
    """
    SalUn (Saliency-based Unlearning) Method.
    Two-phase training with saliency-masked gradient updates.
    """
    criterion = nn.CrossEntropyLoss()
    remain_classes = [i for i in range(NUM_CLASSES) if i != forget_class]
    
    saliency_mask = compute_gradient_saliency(model, forget_loader, criterion, device, saliency_threshold)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    def apply_saliency_mask():
        with torch.no_grad():
            for name, param in model.named_parameters():
                if name in saliency_mask and param.grad is not None:
                    param.grad *= saliency_mask[name]
    
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        total_batches = 0
        
        # Phase 1: Forget data with random labels
        for inputs, labels in forget_loader:
            inputs = inputs.to(device)
            random_labels = torch.tensor([
                remain_classes[torch.randint(0, len(remain_classes), (1,)).item()]
                for _ in range(len(labels))
            ], device=device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, random_labels)
            loss.backward()
            apply_saliency_mask()
            if grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
            running_loss += loss.item()
            total_batches += 1
        
        # Phase 2: Retain data normally
        for inputs, labels in retain_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            apply_saliency_mask()
            if grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
            running_loss += loss.item()
            total_batches += 1
        
        print(f"  Epoch [{epoch+1}/{epochs}] Avg Loss: {running_loss/total_batches:.4f}")
    
    return model

## Load Data

In [ ]:
# Load data once for all experiments
print("Loading CIFAR-10 data...")
train_loader, test_loader, retain_loader, forget_loader, train_set, test_set = \
    get_data_loaders(BATCH_SIZE, FORGET_CLASS)

print("Preparing UMAP subset...")
umap_subset, umap_loader, selected_indices = get_umap_subset(train_set, test_set)

print(f"\nDataset sizes:")
print(f"  Train: {len(train_loader.dataset):,}")
print(f"  Test: {len(test_loader.dataset):,}")
print(f"  Retain: {len(retain_loader.dataset):,}")
print(f"  Forget: {len(forget_loader.dataset):,}")

## Run All Methods on All Models

In [ ]:
# Define models and methods
models_config = [
    ("ResNet-18", get_resnet18),
    ("VGG-16-BN", get_vgg16bn)
]

methods_config = [
    ("RandomLabeling", lambda m, rl, fl, fc, e, lr, d: random_labeling_unlearn(
        m, rl, fl, fc, e, lr, d, MOMENTUM, WEIGHT_DECAY)),
    ("GradientAscent", lambda m, rl, fl, fc, e, lr, d: gradient_ascent_unlearn(
        m, fl, e, lr, d, MAX_GRAD_NORM, MOMENTUM, WEIGHT_DECAY)),
    ("SalUn", lambda m, rl, fl, fc, e, lr, d: salun_unlearn(
        m, rl, fl, fc, e, lr, d, SALIENCY_THRESHOLD, MAX_GRAD_NORM, MOMENTUM, WEIGHT_DECAY))
]

all_results = []

for model_name, model_fn in models_config:
    for method_name, method_fn in methods_config:
        print(f"\n{'='*70}")
        print(f"Running {method_name} on {model_name}")
        print(f"{'='*70}")
        
        # Load fresh pretrained model
        print(f"Loading pretrained {model_name}...")
        model = model_fn().to(device)
        original_model = copy.deepcopy(model)
        
        # Run unlearning
        print(f"Starting {method_name} unlearning...")
        start_time = time.time()
        
        model = method_fn(model, retain_loader, forget_loader, FORGET_CLASS, EPOCHS, LEARNING_RATE, device)
        
        runtime = time.time() - start_time
        print(f"\nUnlearning completed in {runtime:.2f} seconds")
        
        # Generate results
        print(f"Generating results...")
        result = create_results_json(
            model=model,
            train_loader=train_loader,
            test_loader=test_loader,
            umap_subset=umap_subset,
            umap_loader=umap_loader,
            selected_indices=selected_indices,
            forget_class=FORGET_CLASS,
            method_name=method_name,
            model_name=model_name,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            learning_rate=LEARNING_RATE,
            runtime=runtime,
            device=device,
            original_model=original_model
        )
        
        # Save results
        save_results(result, model, output_dir="notebook_results")
        all_results.append(result)
        
        # Print summary
        print(f"\n{'-'*40}")
        print(f"Results for {model_name} + {method_name}:")
        print(f"  UA: {result['UA']:.3f}  RA: {result['RA']:.3f}")
        print(f"  TUA: {result['TUA']:.3f}  TRA: {result['TRA']:.3f}")
        print(f"  FQS: {result['FQS']}  Runtime: {result['RTE']:.1f}s")
        print(f"{'-'*40}")
        
        # Clean up to free memory
        del model, original_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

## Final Comparison

In [ ]:
# Print comprehensive comparison table
print("\n" + "="*85)
print("COMPARISON: All Unlearning Methods")
print("="*85)
print(f"{'Model':<12} {'Method':<16} {'UA':>7} {'RA':>7} {'TUA':>7} {'TRA':>7} {'FQS':>7} {'Time':>8}")
print("-"*85)

for r in all_results:
    print(f"{r['Model']:<12} {r['Method']:<16} {r['UA']:>7.3f} {r['RA']:>7.3f} {r['TUA']:>7.3f} {r['TRA']:>7.3f} {r['FQS']:>7.4f} {r['RTE']:>7.1f}s")

print("="*85)
print("\nMetric Definitions:")
print("  UA  = Unlearning Accuracy (accuracy on forget class in training set)")
print("  RA  = Remain Accuracy (accuracy on other classes in training set)")
print("  TUA = Test Unlearning Accuracy (accuracy on forget class in test set)")
print("  TRA = Test Remain Accuracy (accuracy on other classes in test set)")
print("  FQS = Forgetting Quality Score (higher = better forgetting)")

In [ ]:
# Summary by method (average across models)
print("\n" + "="*70)
print("SUMMARY BY METHOD (averaged across models)")
print("="*70)

methods = ["RandomLabeling", "GradientAscent", "SalUn"]
for method in methods:
    method_results = [r for r in all_results if r['Method'] == method]
    if method_results:
        avg_ua = sum(r['UA'] for r in method_results) / len(method_results)
        avg_ra = sum(r['RA'] for r in method_results) / len(method_results)
        avg_tua = sum(r['TUA'] for r in method_results) / len(method_results)
        avg_tra = sum(r['TRA'] for r in method_results) / len(method_results)
        avg_fqs = sum(r['FQS'] for r in method_results) / len(method_results)
        avg_rte = sum(r['RTE'] for r in method_results) / len(method_results)
        print(f"{method:<16}: UA={avg_ua:.3f} RA={avg_ra:.3f} TUA={avg_tua:.3f} TRA={avg_tra:.3f} FQS={avg_fqs:.4f} Time={avg_rte:.1f}s")

print("\n" + "="*70)
print("SUMMARY BY MODEL (averaged across methods)")
print("="*70)

models = ["ResNet-18", "VGG-16-BN"]
for model in models:
    model_results = [r for r in all_results if r['Model'] == model]
    if model_results:
        avg_ua = sum(r['UA'] for r in model_results) / len(model_results)
        avg_ra = sum(r['RA'] for r in model_results) / len(model_results)
        avg_tua = sum(r['TUA'] for r in model_results) / len(model_results)
        avg_tra = sum(r['TRA'] for r in model_results) / len(model_results)
        avg_fqs = sum(r['FQS'] for r in model_results) / len(model_results)
        avg_rte = sum(r['RTE'] for r in model_results) / len(model_results)
        print(f"{model:<12}: UA={avg_ua:.3f} RA={avg_ra:.3f} TUA={avg_tua:.3f} TRA={avg_tra:.3f} FQS={avg_fqs:.4f} Time={avg_rte:.1f}s")

## Notes

### Expected Behavior:
- **Lower UA/TUA** = Better forgetting (model no longer recognizes forget class)
- **Higher RA/TRA** = Better retention (model still performs well on other classes)
- **Higher FQS** = Better overall forgetting quality

### Method Characteristics:
1. **Random Labeling**: Gentle approach, may not fully forget but preserves retention well
2. **Gradient Ascent**: Aggressive forgetting, may harm retention (catastrophic forgetting risk)
3. **SalUn**: Balanced approach, targets only relevant weights

### Architectural Differences (Research Question):
- **ResNet-18**: Skip connections may make forgetting more difficult (information preserved across layers)
- **VGG-16**: Sequential architecture may allow more localized forgetting